# Redes Neurais e Deep Learning 08

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn as sk

# Carregando a Base de Dados

https://www.kaggle.com/fedesoriano/stroke-prediction-dataset

In [ ]:
dataset = pd.read_csv(r"D:\Users\Nayan Couto\Cloud Drive\Documentos\Arquivos PDF, PPT, DOC\Ciências de Dados - Anhanguera Ampli\Inteligência Artificial\Redes Neurais e Deep Learning\RNA_Deep_Learning\Curso\datasets\stroke.csv")

dataset

# Pré-processamento da base de dados

1. Remoção de linhas nulas
2. Separação dos dados em X (variáveis independentes) e y (variável dependente)
3. Transformação das variáveis categóricas em numéricas (LabelEncoder e OneHotEncoder)
4. Separação em Teste e Treino
5. Normalização

In [ ]:
dataset[dataset.isnull().any(axis=1)]

In [ ]:
#dataset[dataset.isnull().any(axis=1)] quantas linhas nulas?
dataset = dataset.dropna()

dataset

In [ ]:
x = dataset.iloc[:, 1:-1]
y = dataset.iloc[:, -1]

x

In [ ]:
x.dtypes

In [ ]:
x['smoking_status'].unique()

In [ ]:
from sklearn.preprocessing import LabelEncoder

cols = ['ever_married', 'Residence_type'] #unique pra descobrir quantos rótulos únicos
le = LabelEncoder()
x[cols] = x[cols].apply(le.fit_transform)

x.head(10)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder 

#https://towardsdatascience.com/columntransformer-in-scikit-for-labelencoding-and-onehotencoding-in-machine-learning-c6255952731b
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), ['gender', 'work_type', 'smoking_status'])], 
                       remainder='passthrough')
x = ct.fit_transform(x)

x[0]

In [ ]:
colunas = ['ohe1', 'ohe2', 'ohe3', 'ohe4', 'ohe5', 'ohe6', 'ohe7', 'ohe8', 'ohe9', 'ohe10', 'ohe11', 
           'ohe12', 'age', 'hypertension', 'heart_disease', 'ever_married', 'Residence_type', 
           'avg_glucose_level', 'bmi']
pd.DataFrame(x, columns=colunas)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

x_train

# Arquitetura de Rede Neural (MLP)

In [ ]:
tf.__version__

In [ ]:
from sklearn.model_selection import GridSearchCV
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.wrappers.scikit_learn import KerasClassifier

from tensorflow import keras
from tf_keras.models import Sequential
from tf_keras.layers import Dense
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier

# GridSearch na MLP

https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

In [ ]:
def create_model(optimizer='adam'):
    ann = Sequential()
    ann.add(Dense(units=6, activation='relu', kernel_initializer='he_normal'))
    ann.add(Dense(units=6, activation='relu', kernel_initializer='he_normal'))
    ann.add(Dense(units=1, activation='sigmoid'))
    ann.compiled(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return ann

In [ ]:
# create model
model = KerasClassifier(build_fn=create_model, verbose=2)

In [ ]:
# define the grid search parameters
optimizer = ['SGD', 'Adam']
batch_size = [16, 32, 64]
epochs = [10, 20, 30, 40, 50]
param_grid = dict(optimizer=optimizer, batch_size=batch_size, epochs=epochs)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=None, cv=5)
grid_result = grid.fit(x_train, y_train)

# Resultados

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Fashion MNIST

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn as sk

# Carregando a Base de Dados

https://www.kaggle.com/zalando-research/fashionmnist

In [ ]:
dataset_train = pd.read_csv("fashion_mnist/train.csv")
dataset_test = pd.read_csv("fashion_mnist/test.csv")

dataset_train

# Pré-processamento da base de dados

1. Separação dos dados em X (variáveis independentes) e y (variável dependente)
2. Normalização

In [ ]:
X_train = dataset_train.iloc[:, 1:]
y_train = dataset_train.iloc[:, 0]
X_test = dataset_test.iloc[:, 1:]
y_test = dataset_test.iloc[:, 0]

X_train

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train

# Arquitetura de Rede Neural (MLP)

In [ ]:
tf.__version__

In [ ]:
gpus = tf.config.list_physical_devices('GPU')

gpus

In [ ]:
from sklearn.model_selection import GridSearchCV
from tf_keras.models import Sequential
from tf_keras.layers import Dense
from scikeras.wrappers import KerasClassifier

# GridSearch na MLP

https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

In [ ]:
def create_model(optimizer='adam'):
    ann = Sequential()
    ann.add(Dense(units=128, activation='relu', kernel_initializer='he_normal'))
    ann.add(Dense(units=10, activation='softmax'))
    ann.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', 
                metrics=['accuracy'])
    
    return ann

In [ ]:
# create model
model = KerasClassifier(build_fn=create_model, verbose=1)

In [ ]:
# define the grid search parameters
optimizer = ['SGD', 'Adam']
batch_size = [16, 32]
epochs = [20, 30]
param_grid = dict(optimizer=optimizer, batch_size=batch_size, epochs=epochs)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=None, cv=5)
grid_result = grid.fit(X_train, y_train)

# Resultados

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Otimizando SGD

In [ ]:
def create_model_SGD(neurons=128, learn_rate=0.01, momentum=0):
    ann = Sequential()
    ann.add(Dense(units=neurons, activation='relu', kernel_initializer='he_normal'))
    ann.add(Dense(units=10, activation='softmax'))
    optimizer = tf.keras.optimizers.SGD(learning_rate=learn_rate, momentum=momentum)
    ann.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return ann

# create model
model = KerasClassifier(build_fn=create_model_SGD, epochs=30, batch_size=32, verbose=2)

In [ ]:
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1]
momentum = [0.0, 0.2, 0.4]
neurons = [64, 128]
param_grid = dict(neurons=neurons, learn_rate=learn_rate, momentum=momentum)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=None, cv=5)
grid_result = grid.fit(X_train, y_train)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Save best model
https://machinelearningmastery.com/save-load-keras-deep-learning-models/

In [ ]:
best_model = Sequential()
best_model.add(Dense(units=128, activation='relu', kernel_initializer='he_normal'))
best_model.add(Dense(units=10, activation='softmax'))

opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0)
best_model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = best_model.fit(X_train, y_train, epochs=30, batch_size=32)

In [ ]:
loss, accuracy = best_model.evaluate(X_test, y_test)
print('Loss: %.2f' % loss)
print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
best_model.predict(X_test)[0]

In [ ]:
model_json = best_model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

best_model.save_weights("model.h5")
print("Saved model to disk")

# Load model

In [ ]:
from tf_keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model.h5")

print("Loaded model from disk")